In [4]:
import numpy as np
import pandas as pd
import torch

In [5]:
def set_seed(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

seed = 1337
set_seed(seed)

In [6]:
import os
import sys

notebook_dir = os.getcwd()

root_dir = os.path.abspath(os.path.join(notebook_dir, "../../"))
sys.path.append(root_dir)

In [12]:
from dotenv import load_dotenv

load_dotenv()

from service import EnvironmentSecretService
from coral import DataSourceFactory
from core.models.lookback import Lookback
from core.models.timeframe import Timeframe
from core.models.datasource_type import DataSourceType
from core.models.protocol_type import ProtocolType

In [9]:
DEFAULT_DATASOUCE = DataSourceType.BYBIT
DEFAULT_TIMEFRAME = Timeframe.FIFTEEN_MINUTES
DEFAULT_LOOKBACK = Lookback.EIGHT_MONTH

In [11]:
datasource = DataSourceFactory(EnvironmentSecretService())
datasource.register_rest_exchange(DEFAULT_DATASOUCE)

In [17]:
ds = datasource.create(DEFAULT_DATASOUCE, ProtocolType.REST)

symbols = ds.fetch_future_symbols()

ob = ds.fetch_order_book(symbols[0])
bids, asks = np.array(ob[0]), np.array(ob[1])

bids.shape

(30, 2)